In [ ]:
import ray
from ray.rllib.algorithms.ppo import PPOConfig
from ray.rllib.policy.policy import PolicySpec
from ray.tune import register_env

import rllib_setup

ray.init()
env_name = "VJS"
register_env(
    env_name,
    lambda config: rllib_setup.get_env_continuous(),
)
test_env = rllib_setup.get_env_continuous()


def policies(agent_ids):
    obs_space = test_env.observation_space
    act_space = test_env.action_space
    return {
        str(i): (
            None,
            obs_space,
            act_space,
            {}
            # config=config.overrides(agent_id=int(i[8:])),
        )
        for i in agent_ids
    }


config = (
    PPOConfig().rollouts(num_rollout_workers=10)
    .training(gamma=0.9, lr=0.01) 
    .resources(num_gpus=1)
    .multi_agent(
        policies=policies(test_env._agent_ids),
        policy_mapping_fn=lambda agent_id, episode, **kwargs: str(agent_id),
    )
    .environment(disable_env_checking=True)
)
print(config.to_dict())
# Build a Algorithm object from the config and run one training iteration.
algo = config.build(env=env_name)

In [4]:
for j in range(10):
    for i in range(10):
        info = algo.train()
        info['info']['learner']['Machine_6']['learner_stats']['total_loss']

(raylet) [2023-09-29 11:19:28,025 E 3473473 3473473] (raylet) node_manager.cc:3069: 4 Workers (tasks / actors) killed due to memory pressure (OOM), 0 Workers crashed due to other reasons at node (ID: 9648b9014e4abaede66aee64b1344246a1b2b8bbe34248617d11f84d, IP: 192.168.3.6) over the last time period. To see more information about the Workers killed on this node, use `ray logs raylet.out -ip 192.168.3.6`
(raylet) 
(raylet) Refer to the documentation on how to address the out of memory issue: https://docs.ray.io/en/latest/ray-core/scheduling/ray-oom-prevention.html. Consider provisioning more memory on this node or reducing task parallelism by requesting more CPUs per task. To adjust the kill threshold, set the environment variable `RAY_memory_usage_threshold` when starting Ray. To disable worker killing, set the environment variable `RAY_memory_monitor_refresh_ms` to zero.


ValueError: Expected parameter loc (Tensor of shape (46, 1)) of distribution Normal(loc: torch.Size([46, 1]), scale: torch.Size([46, 1])) to satisfy the constraint Real(), but found invalid values:
tensor([[nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan]], device='cuda:0', grad_fn=<SplitBackward0>)
 tracebackTraceback (most recent call last):
  File "/home/yuan/ResM

(raylet) [2023-09-29 11:21:28,027 E 3473473 3473473] (raylet) node_manager.cc:3069: 1 Workers (tasks / actors) killed due to memory pressure (OOM), 0 Workers crashed due to other reasons at node (ID: 9648b9014e4abaede66aee64b1344246a1b2b8bbe34248617d11f84d, IP: 192.168.3.6) over the last time period. To see more information about the Workers killed on this node, use `ray logs raylet.out -ip 192.168.3.6`
(raylet) 
(raylet) Refer to the documentation on how to address the out of memory issue: https://docs.ray.io/en/latest/ray-core/scheduling/ray-oom-prevention.html. Consider provisioning more memory on this node or reducing task parallelism by requesting more CPUs per task. To adjust the kill threshold, set the environment variable `RAY_memory_usage_threshold` when starting Ray. To disable worker killing, set the environment variable `RAY_memory_monitor_refresh_ms` to zero.
(raylet) [2023-09-29 11:22:28,028 E 3473473 3473473] (raylet) node_manager.cc:3069: 9 Workers (tasks / actors) kill